In [12]:
import numpy as np
import pandas as pd
%run ../src/models/geo_helpers.py
%run ../src/models/section_series.py
%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format

/usr/local/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [13]:
series_loader = SectionSeriesLoader(
    img_shape=(200, 200),
    SRC_PATH='../data/interim/Version_4_DMSP-OLS_Nighttime_Lights_Time_Returns/*.npz')

In [149]:
city_locations_path = '../data/external/' +\
    'worldcitiespop.txt'

city_locations = pd.read_csv(city_locations_path, encoding = "ISO-8859-1").rename(columns={'City': 'city'})
city_locations.head()

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Country,city,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,6.0000,nan,42.4833,1.4667
1,ad,aixirivali,Aixirivali,6.0000,nan,42.4667,1.5000
2,ad,aixirivall,Aixirivall,6.0000,nan,42.4667,1.5000
3,ad,aixirvall,Aixirvall,6.0000,nan,42.4667,1.5000
4,ad,aixovall,Aixovall,6.0000,nan,42.4667,1.4833


In [150]:
earthquake_locations_path = '../data/external/' +\
    'earthquakes_affected_locations/earthquakes_affected_locations.csv'

earthquake_locations = pd.read_csv(src_earthquakes_path, encoding = "ISO-8859-1")
earthquake_locations['city'] = earthquake_locations['city'].apply(lambda c: c.lower())
earthquake_locations.head()

,id,country,city,latitude,longitude,geoname,year,month,day,Totaldeaths,Totalaffected,Totaldamage000US,insured_losses
0,1,Afghanistan,takhar,36.7000,69.8000,NaN,1992,5,20,14,0,0,0
1,2,Afghanistan,jowzan,36.7500,66.0000,Jowzjan,1994,5,1,160,100330,0,0
2,2,Afghanistan,mazar-i-sharif,36.7500,67.0000,mazar-i-sharif,1994,5,1,160,100330,0,0
3,2,Afghanistan,balkh,36.7501,66.8997,balkh,1994,5,1,160,100330,0,0
4,2,Afghanistan,termez,37.2242,67.2783,NaN,1994,5,1,160,100330,0,0


In [160]:
%%time
annual_frames = {}
for year in earthquake_locations.year.unique()[:1]:
    annual_frames[year] = pd.merge(city_locations, earthquake_locations[earthquake_locations.year == year], on='city', how='left')
    annual_frames[year]['year'] = year
    annual_frames[year].fillna(0).reset_index().groupby('index').agg({
        'Totaldeaths': 'sum',
        'Totalaffected': 'sum',
        'Totaldamage000US': 'sum',
        'insured_losses': 'sum',
        'Latitude': 'first',
        'Longitude': 'first'
    })

CPU times: user 5.22 s, sys: 4.12 s, total: 9.35 s
Wall time: 11.3 s
